In [ ]:
from bs4 import BeautifulSoup as bs
import requests as req
from pprint import pprint
import re
import os

In [ ]:
# 這個URL是所有中文書的列表
url = 'https://www.gutenberg.org/browse/languages/zh'

# 建立一個list存放等等要用的東西
list_posts=[]

#不知道需不需要headers才能爬 總之試試看
headers = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/115.0 Safari/537.36"
}

res = req.get(url, headers=headers)
soup = bs(res.text, 'lxml')

In [ ]:
list_posts.clear()
seen_titles = set() # 創一個set記住已經看過的title，過濾完全一樣名子的書
for a in soup.select('div.page_content li.pgdbetext a'):
    title = a.get_text()
    if title not in seen_titles: # 檢查title是否有在seen_title裡面
        list_posts.append({ # 沒有就append
            'title': title,
            'link': 'https://www.gutenberg.org' + a['href'] + '.txt.utf-8'
            })
        seen_titles.add(title)
seen_titles.clear()
pprint(list_posts)

In [ ]:
for index, obj in enumerate(list_posts):

    ### 由於我使用的網址是純文字網址，無法get element，直接用text就好!!!! ###
    # res_ = req.get(obj['link']) # 從我們的list_posts這個dict中 取出link
    # soup_ = bs(res_.text, 'lxml')# 把取出的link去做解讀~
    # text_block = soup_.select_one('pre') # 抓<pre>
    # if text_block:
    #     raw_text = text_block.get_text() # 去html標籤
    # else:
    #     raw_text = ''
     ### 由於我使用的網址是純文字網址，無法get element，直接用text就好!!!! ###

    res_ = req.get(obj['link'])
    raw_text = res_.text

    chinese_text = ''.join(re.findall(r'[\u4e00-\u9fff\，\。\？\！\「\」\《\》\；\：\『\』]+', raw_text))
    # 用正規表達式只抓中文!這邊比較土炮，手補全型符號
    
    #把抓到的文字新增到list_posts裡面 key='text'
    obj['text']= chinese_text
    print(f"list_posts已新增{obj['title']}的內容")

    # 測試用
    # if index == 2:
    #     break

pprint(list_posts)


In [ ]:
os.mkdir("project_gutenberg")
output_dir = "project_gutenberg"

for item in list_posts:
    if item['text'] != '': # 因為有三本書是純英文，經過只抓中文、全型符號的正規表達式之後會變成三本空白的txt，所以我們這邊過濾
        title = item['title']
        text = item.get('text','')
        
        # 避免檔名有非法字元！必須先清理title，因為有幾本書竟然是英文名子，含空白，害我白跑
        safe_title = "".join(c for c in title if c.isalnum() or c in " _-")
        # 定義file_path
        file_path = os.path.join("project_gutenberg", f"{safe_title}.txt")
        # 建立檔案並寫入文字
        with open(file_path, "w", encoding="utf-8") as f:
            f.write(text)